In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os
from typing import Dict, List, Tuple
from collections import defaultdict

In [2]:
def load_model(model_name):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        # use_flash_attention_2=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

model, tokenizer = load_model("deepseek-ai/deepseek-moe-16b-base")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [19]:
def get_top_k_tokens(hidden_states: torch.Tensor, lm_head: torch.nn.Linear, tokenizer, k: int = 5) -> List[Tuple[str, float]]:
    """ get topk tokens from hidden states using lm head """
    with torch.no_grad():
        # Ensure hidden_states has at least 2 dimensions (batch_size, num_tokens, hidden_dim)
        if hidden_states.dim() == 2:
            hidden_states = hidden_states.unsqueeze(0)  # Add batch dimension
            # Apply RMS normalization like in DeepseekRMSNorm
            variance = hidden_states.pow(2).mean(-1, keepdim=True) 
            hidden_states = hidden_states * torch.rsqrt(variance + 1e-6)

        logits = lm_head(hidden_states)  # (batch_size, num_tokens, vocab_size)
    
    # Get top-k tokens
    scores, token_ids = torch.topk(logits, k=k, dim=-1)  # (batch_size, num_tokens, k)
    
    # Decode tokens and collect results for each position
    results = []
    for pos in range(scores.size(1)):  # Iterate over token positions
        pos_results = []
        for i in range(k):
            token = tokenizer.decode(token_ids[0, pos, i])  # Decode token for this position
            score = scores[0, pos, i].item()  # Get score for this position
            pos_results.append((token, score))
        results.append(pos_results)
    
    return results

In [33]:
class DeepseekLayerAnalyzer:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.layer_outputs = defaultdict(list)
        self.moe_gate_outputs = defaultdict(list)
        self.moe_combined_outputs = defaultdict(list)
        self.expert_outputs = defaultdict(lambda: defaultdict(list))
        self.shared_expert_outputs = defaultdict(list)
        self.hooks = []
        
    def register_hooks(self):
        """Register hooks for layer outputs and MoE combination points"""
        
        def layer_output_hook(layer_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing layer outputs"""
                hidden_states = outputs[0] if isinstance(outputs, tuple) else outputs
                self.layer_outputs[layer_idx].append(hidden_states.detach())
            return hook

        def moe_gate_hook(layer_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing MoE gate outputs before expert computation"""
                # Capture topk_idx, topk_weight, and aux_loss from gate outputs
                if isinstance(outputs, tuple):
                    topk_idx, topk_weight, aux_loss = outputs
                    self.moe_gate_outputs[layer_idx].append({
                        'topk_idx': topk_idx.detach(),
                        'topk_weight': topk_weight.detach(),
                        'aux_loss': aux_loss.detach() if aux_loss is not None else None
                    })
            return hook

        def expert_hook(layer_idx, expert_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing expert outputs"""
                # Get the latest gate outputs for this layer
                if not self.moe_gate_outputs[layer_idx]:
                    return print(f'no gate outputs for layer {layer_idx}')
            
                gate_data = self.moe_gate_outputs[layer_idx][-1]
                
                # Handle 2D or 3D tensor shapes
                if len(gate_data['topk_idx'].shape) == 2:
                    batch_size = 1
                    seq_len, top_k = gate_data['topk_idx'].shape
                else:
                    batch_size, seq_len, top_k = gate_data['topk_idx'].shape
                
                # Get mask for tokens where this expert was selected
                expert_mask = (gate_data['topk_idx'] == expert_idx)                
                # Flatten and find positions where this expert was selected
                selected_positions = torch.nonzero(expert_mask, as_tuple=True)
                # If no tokens selected this expert, skip
                if selected_positions[0].numel() == 0:
                    return
                    
                # Get the actual inputs routed to this expert
                # Inputs[0] shape: (total_selected_tokens, hidden_dim)
                total_selected = inputs[0].shape[0] 
                # Validate we're processing the correct number of tokens
                expected_selected = expert_mask.sum().item()
                if total_selected != expected_selected:
                    print(f" expert {expert_idx} processed {total_selected} tokens but expected {expected_selected}")
                    return
                    
                # Get the full hidden states from outputs
                # outputs shape: (total_selected_tokens, hidden_dim)
                hidden_states = outputs
                if isinstance(outputs, tuple):
                    hidden_states = outputs[0]
                    
                # Record data for each selected position
                for pos_idx, pos in enumerate(selected_positions[0]):
                    token_data = {
                        'position': pos.item(),
                        'input': inputs[0][pos_idx].detach(),
                        'output': outputs[pos_idx].detach(),
                        'hidden_state': hidden_states[pos_idx].detach()  # Store full hidden state
                    }
                    
                    # Get the corresponding gate weight for this position
                    # Find which expert slot (in top_k) this expert was selected for this position
                    expert_slots = (gate_data['topk_idx'][pos.item()] == expert_idx).nonzero(as_tuple=True)[0]
                    if len(expert_slots) > 0:
                        token_data['gate_weight'] = gate_data['topk_weight'][pos.item()][expert_slots[0]].item()
                    
                    self.expert_outputs[layer_idx][expert_idx].append(token_data)
            return hook
        
        def shared_expert_hook(layer_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing shared expert outputs"""
                self.shared_expert_outputs[layer_idx].append({
                    'input': inputs[0].detach(),
                    'output': outputs.detach()
                })
            return hook

        def moe_combine_hook(layer_idx):
            def hook(module, inputs, outputs):
                """Hook for capturing final combined MoE outputs"""
                # For DeepseekMoE, this captures the weighted sum of expert outputs
                self.moe_combined_outputs[layer_idx].append({
                    'combined_output': outputs.detach(),
                    'input': inputs[0].detach()  # Original input before MoE
                })
            return hook

        # Register hooks for each layer
        for layer_idx, layer in enumerate(self.model.model.layers):
            # Hook for layer output
            hook = layer.register_forward_hook(layer_output_hook(layer_idx))
            self.hooks.append(hook)
            
            # If it's an MoE layer, add MoE-specific hooks
            if hasattr(layer.mlp, 'experts'):
                # Hook for gate mechanism
                gate_hook = layer.mlp.gate.register_forward_hook(moe_gate_hook(layer_idx))
                self.hooks.append(gate_hook)
                
                # Hook for each expert
                for expert_idx, expert in enumerate(layer.mlp.experts):
                    expert_hook_fn = expert.register_forward_hook(expert_hook(layer_idx, expert_idx))
                    self.hooks.append(expert_hook_fn)

                # Hook for shared expert if it exists
                if hasattr(layer.mlp, 'shared_experts'):
                    shared_hook = layer.mlp.shared_experts.register_forward_hook(shared_expert_hook(layer_idx))
                    self.hooks.append(shared_hook)
                
                # Hook for final combined output
                combine_hook = layer.mlp.register_forward_hook(moe_combine_hook(layer_idx))
                self.hooks.append(combine_hook)

    def analyze_tokens(self, input_ids: torch.Tensor, return_hidden_states: bool = False) -> Dict:
        """ run inference and analyze tokens at each layer and expert combination point """

        self.moe_combined_outputs.clear()
        self.expert_outputs.clear()
        self.shared_expert_outputs.clear()
        
        # Forward pass
        with torch.no_grad():
            outputs = self.model(input_ids)
        
        results = {
            'layer_predictions': {},
            'moe_analysis': {},
            'hidden_states': {} if return_hidden_states else None
        }
        
        # Analyze layer outputs
        for layer_idx, outputs in self.layer_outputs.items():
            if not outputs:  # Skip if no outputs captured
                continue
            hidden_states = outputs[-1]  # Get last captured output
            
            
            
            
            # Get token predictions for this layer
            top_tokens = get_top_k_tokens(hidden_states, self.model.lm_head, self.tokenizer)
            results['layer_predictions'][layer_idx] = top_tokens
            
            if return_hidden_states:
                results['hidden_states'][f'layer_{layer_idx}'] = hidden_states
        
        # Analyze MoE layers
        for layer_idx in self.moe_gate_outputs.keys():
            if not self.moe_gate_outputs[layer_idx]:
                continue
                
            gate_data = self.moe_gate_outputs[layer_idx][-1]  # Get last captured data
            combined_data = self.moe_combined_outputs[layer_idx][-1]
            
            # Initialize predictions dictionary by position
            expert_predictions_by_pos = defaultdict(dict)
            expert_hidden_states_by_pos = defaultdict(dict)
            
            # Process expert outputs by position
            for expert_idx, data_list in self.expert_outputs[layer_idx].items():
                for data in data_list:
                    position = data['position']
                    # Get predictions for this expert's output at this position
                    predictions = get_top_k_tokens(
                        data['output'].unsqueeze(0),
                        self.model.lm_head,
                        self.tokenizer
                    )
                    expert_predictions_by_pos[position][expert_idx] = predictions[0]  # [0] because we only have one prediction set

                    # Store hidden states
                    expert_hidden_states_by_pos[position][expert_idx] = {
                    'hidden_state': data['hidden_state'].tolist(),
                    'gate_weight': data.get('gate_weight', None)
                }
            
            # Get predictions for shared expert if it exists
            if self.shared_expert_outputs[layer_idx]:
                shared_expert_predictions = get_top_k_tokens(
                    self.shared_expert_outputs[layer_idx][-1]['output'],
                    self.model.lm_head,
                    self.tokenizer
                )
            
            # Update experts_analysis dictionary to include hidden states
            experts_analysis = {
            'selected_experts': gate_data['topk_idx'].tolist(),
            'expert_weights': gate_data['topk_weight'].tolist(),
            'aux_loss': gate_data['aux_loss'].item() if gate_data['aux_loss'] is not None else None,
            'expert_predictions_by_position': dict(expert_predictions_by_pos),
            'expert_hidden_states_by_position': dict(expert_hidden_states_by_pos),
            'shared_expert_predictions': shared_expert_predictions if self.shared_expert_outputs[layer_idx] else None
        }
            
            # Get token predictions from combined output
            combined_tokens = get_top_k_tokens(
                combined_data['combined_output'], 
                self.model.lm_head,
                self.tokenizer
            )
            
            experts_analysis['combined_output_tokens'] = combined_tokens
            results['moe_analysis'][layer_idx] = experts_analysis

        return results
    
    def cleanup(self):
        """remove all registered hooks"""
        for hook in self.hooks:
            hook.remove()
        self.hooks.clear()

def analyze_deepseek_moe(model, tokenizer, input_text: str, return_hidden_states: bool = False):
    """ analyze DeepSeek MoE model behavior for given input text """
    analyzer = DeepseekLayerAnalyzer(model, tokenizer)
    analyzer.register_hooks()
    
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    
    try:
        results = analyzer.analyze_tokens(input_ids, return_hidden_states=return_hidden_states)
        return results
    finally:
        analyzer.cleanup()  

In [34]:
prompt = '''The mystery of life isn't a problem to solve, but a reality to'''

analysis = analyze_deepseek_moe(
    model, 
    tokenizer,
    input_text=prompt
)

In [35]:
input_ids = tokenizer(text="venir", return_tensors="pt",).input_ids
print(f'input_ids : {input_ids}')

input_ids : tensor([[100000,  46310]])


In [36]:
# layer_idx = 1  # example layer
# token_pos = 4
# expert_idx = 3
# hidden_state_list = analysis['moe_analysis'][layer_idx]['expert_hidden_states_by_position'][token_pos][expert_idx]['hidden_state']
# print(f'hidden_state_list : {len(hidden_state_list)}')


In [37]:
def get_post_attn_ln_inputs(model, tokenizer, text):
    """places a hook on the post attention layernorm to retrieve its inputs"""
    # Store inputs in a dict mapping layer idx -> inputs
    post_attn_ln_inputs = {}
    hooks = []
    
    def hook_post_attn_ln(module, input, output, layer_idx):
        if layer_idx not in post_attn_ln_inputs:
            post_attn_ln_inputs[layer_idx] = []
        post_attn_ln_inputs[layer_idx].append([x.detach() for x in input])
    
    # Register hooks on post attention layernorm for each layer
    for i, layer in enumerate(model.model.layers):
        hooks.append(
            layer.post_attention_layernorm.register_forward_hook(
                lambda m, i, o, idx=i: hook_post_attn_ln(m, i, o, idx)
            )
        )
    
    try:
        # Run inference
        input_ids = tokenizer(text, return_tensors="pt").input_ids
        model(input_ids)
        
        return post_attn_ln_inputs
        
    finally:
        # Clean up hooks
        for hook in hooks:
            hook.remove()

In [38]:
post_attn_ln_inputs = get_post_attn_ln_inputs(model, tokenizer, text=prompt)

In [39]:
def verify_moe_equation(model, tokenizer, layer_idx, text="the quick brown fox", tolerance=1e-5):
    """
    Verifies that MoE activations add up correctly:
    final_layer_output = residual_stream_hidden_states_before_experts + moe_output
    """
    analyzer = DeepseekLayerAnalyzer(model, tokenizer)
    analyzer.register_hooks()
    
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    try:
        # Run the analysis to populate data structures
        analyzer.analyze_tokens(input_ids)
        # Get moe outputs and residual
        if layer_idx not in analyzer.moe_combined_outputs or not analyzer.moe_combined_outputs[layer_idx]:
            print("No combined output data found")
            return False

        residual = post_attn_ln_inputs[layer_idx][-1][0][0]
        print(f"Residual shape: {residual.shape}")
        
        # Get the combined MoE output directly from the hook
        moe_output = analyzer.moe_combined_outputs[layer_idx][-1]['combined_output'][0]
        print(f"MoE output shape: {moe_output.shape}")

        # Get the final layer output
        final_layer_output = analyzer.layer_outputs[layer_idx][-1][0]
        print(f"Final layer output shape: {final_layer_output.shape}")
        
        # Full equation: final_output = residual + moe_output
        lhs = residual + moe_output
        print(f"lhs shape: {lhs.shape}")
        print(f'lhs {lhs}')
        rhs = final_layer_output
        print(f"rhs shape: {rhs.shape}")
        print(f'rhs {rhs}')
        # Check if close
        is_close = torch.allclose(lhs, rhs, rtol=tolerance, atol=tolerance)
        print(f'is_close {is_close}')
        if not is_close:
            max_diff = (lhs - rhs).abs().max().item()
            print(f"Maximum difference: {max_diff:.6f}")
            print("\nDetailed component analysis:")
            print(f"Residual max value: {residual.abs().max().item():.6f}")
            print(f"MoE output max value: {moe_output.abs().max().item():.6f}")
            print(f"Final output max value: {final_layer_output.abs().max().item():.6f}")
            
        return is_close
        
    finally:
        analyzer.cleanup()

In [40]:
# # Verify equation for a specific layer
# is_valid = verify_moe_equation(model, tokenizer, layer_idx=26, )
# print(f"MoE equation holds: {is_valid}")

In [46]:
# Access layer predictions
for layer_idx, preds in analysis['layer_predictions'].items():
    print(f"Layer {layer_idx} predictions:", preds)

# Access MoE analysis
for layer_idx, moe_data in analysis['moe_analysis'].items():
    print(f"\nMoE Layer {layer_idx}:")
    print("Selected experts:", moe_data['selected_experts'])
    print("Expert weights:", moe_data['expert_weights'])
    print("Combined output tokens:", moe_data['combined_output_tokens'])
    print("Expert predictions by position:", moe_data['expert_predictions_by_position'])
    print("Shared expert predictions:", moe_data['shared_expert_predictions'])

Layer 0 predictions: [[('开辟', 1.318359375), ('晗', 1.244140625), ('wk', 1.2431640625), ('始终坚持', 1.2177734375), ('CM', 1.2080078125)], [('ODO', 1.1943359375), ('吸取', 1.1494140625), ('AGA', 1.123046875), ('辜负', 1.0869140625), ('ADO', 1.078125)], [('щ', 1.021484375), ('CELL', 0.97705078125), ('abla', 0.943359375), ('自我', 0.94287109375), ('ennes', 0.9111328125)], [('wk', 0.70263671875), ('稳步', 0.68701171875), ('UA', 0.6787109375), ('懿', 0.66845703125), ('充分利用', 0.66552734375)], [('wk', 0.88330078125), ('igen', 0.7978515625), ('jak', 0.767578125), ('Wise', 0.74658203125), ('WL', 0.74267578125)], [('JA', 0.7861328125), ('加快推进', 0.7412109375), ('QT', 0.73974609375), ('KF', 0.73486328125), ('UA', 0.728515625)], [('钰', 0.70166015625), ('建立起', 0.66943359375), ('欣喜', 0.65576171875), ('一再', 0.654296875), ('固执', 0.65283203125)], [('-', 0.603515625), ('ca', 0.57763671875), ('se', 0.5693359375), ('work', 0.568359375), (' h', 0.55029296875)], [('设法', 0.796875), ('加快推进', 0.76708984375), ('晖', 0.75341796

In [ ]:
def get_layer_predictions_for_token(results: dict, layer_idx: int, token_pos: int) -> list:
    # Ensure the layer exists in the results
    if layer_idx not in results['layer_predictions']:
        raise ValueError(f"Layer {layer_idx} not found in the results.")

    # Get the layer predictions for the specified token position
    layer_predictions = results['layer_predictions'][layer_idx][token_pos]
    
    # Return top 5 predictions
    return layer_predictions[:5]

def get_shared_expert_predictions_for_token(results: dict, layer_idx: int, token_pos: int) -> list:
    # Ensure the layer exists in the results
    if layer_idx not in results['moe_analysis']:
        raise ValueError(f"Layer {layer_idx} not found in the results.")

    # Get the MoE analysis for the specified layer
    moe_analysis = results['moe_analysis'][layer_idx]

    # Get the shared expert predictions for the specified token position
    shared_predictions = moe_analysis['shared_expert_predictions'][token_pos]
    
    # Return top 5 predictions
    return shared_predictions[:5]

def get_expert_preds(results: dict, layer_idx: int, token_pos: int) -> list:
    # Ensure the layer exists in the results
    if layer_idx not in results['moe_analysis']:
        raise ValueError(f"Layer {layer_idx} not found in the MoE analysis results.")
        
    # Get the MoE analysis for the specified layer
    expert_preds = []
    moe_analysis = results['moe_analysis'][layer_idx]
    expert_predictions = moe_analysis['expert_predictions_by_position']
    expert_toks = expert_predictions[token_pos]

    for expert_idx, preds in expert_toks.items():
        print(f'expert {expert_idx} : {preds}')
        expert_preds.append((int(expert_idx), preds))

    expert_preds = sorted(expert_preds, key=lambda x: x[0])
        
    return expert_preds

In [48]:
def get_highest_pred(results: dict, model, tokenizer, layer_idx: int, token_pos: int, k: int = 5) -> list:
    """gets predictions from expert with highest weight after adding residual """
    # Ensure layer exists in results
    if layer_idx not in results['moe_analysis']:
        raise ValueError(f"Layer {layer_idx} not found in the MoE analysis results.")
        
    # Get MoE analysis for the layer
    moe_analysis = results['moe_analysis'][layer_idx]
    
    # Get expert weights for this token
    expert_weights = {}
    selected_experts = moe_analysis['selected_experts'][token_pos]
    expert_weights_list = moe_analysis['expert_weights'][token_pos]
    
    if isinstance(selected_experts, int):
        # Handle case where only one expert is selected
        expert_weights[selected_experts] = 1.0
    else:
        # Handle case where multiple experts are selected with weights
        for expert_idx, weight in zip(selected_experts, expert_weights_list):
            expert_weights[expert_idx] = weight
                
    # Find expert with highest weight
    # max_weight_expert = max(expert_weights.items(), key=lambda x: x[1])[0]
    max_weight_expert = sorted(expert_weights.items(), key=lambda x: x[1], reverse=True)[k][0]
    print(f'max_weight_expert {max_weight_expert}')
    
    # Get hidden state from this expert
    expert_hidden_state = moe_analysis['expert_hidden_states_by_position'][token_pos][max_weight_expert]['hidden_state']
    
    # Get residual stream
    residual = post_attn_ln_inputs[layer_idx][-1][0][0][token_pos]
    
    # Convert expert hidden state to tensor if it's a list
    if isinstance(expert_hidden_state, list):
        expert_hidden_state = torch.tensor(expert_hidden_state, dtype=torch.float16)
    
    # Add residual to expert hidden state
    combined = residual + expert_hidden_state
    
    # Get logits
    logits = model.lm_head(combined.unsqueeze(0))
    
    # Get top 5 predictions
    topk = torch.topk(logits[0], k=5)
    scores = topk.values.tolist()
    tokens = topk.indices.tolist()
    
    # Convert to token strings
    predictions = []
    for score, token in zip(scores, tokens):
        token_str = tokenizer.decode(token)
        predictions.append((token_str, score))
        
    return predictions

In [49]:
n = len(tokenizer.encode(prompt))
print(f'n : {n}')
layer_idx = 16
token_pos = n-1
k = 0 # which max expert to use
get_highest_pred(analysis, model, tokenizer, layer_idx=layer_idx, token_pos=token_pos, k=k)

n : 17
max_weight_expert 28


[('cades', 9.3671875),
 ('cesi', 9.1796875),
 ('iats', 8.7265625),
 ('roduction', 8.5625),
 ('apic', 7.921875)]

In [50]:
layer_idx = 25
token_pos = 2
layer_preds = get_layer_predictions_for_token(analysis, layer_idx=layer_idx, token_pos=token_pos)
shared_preds = get_shared_expert_predictions_for_token(analysis, layer_idx=layer_idx, token_pos=token_pos)
expert_preds = get_expert_preds(analysis, layer_idx=layer_idx, token_pos=token_pos)

print(f'layer_preds : {layer_preds}')
print(f'shared_preds : {shared_preds}')
print(f'expert_preds : {expert_preds}')

expert 13 : [(' application', 10.9140625), (' Application', 10.1171875), ('application', 9.28125), ('fono', 8.9453125), (' applications', 8.6171875)]
expert 25 : [('BLOCK', 10.0234375), ('board', 9.9140625), (' Slip', 9.7578125), ('blocking', 9.03125), ('plum', 8.8671875)]
expert 37 : [(' con', 20.609375), (' en', 13.453125), ('Con', 12.203125), (' Con', 12.1796875), ('con', 12.0625)]
expert 39 : [('последствие', 11.8671875), (' had', 11.8515625), ('had', 11.4140625), ('Had', 10.78125), (' Кън', 10.7734375)]
expert 51 : [('rob', 10.25), ('is', 9.8203125), (' reveals', 9.578125), ('каза', 9.3046875), (' tells', 9.2578125)]
expert 52 : [('couple', 13.140625), (' couple', 12.6171875), ('sus', 10.8671875), ('íř', 10.4140625), ('openg', 10.1796875)]
layer_preds : [('surround', 22.0), ('�乐', 21.875), ('TOOLSET', 18.96875), (' Surround', 18.546875), (' изчезване', 17.625)]
shared_preds : [(',', 4.48046875), ('\n', 3.87109375), (' ', 3.7890625), ('.', 3.24609375), (' -', 2.921875)]
expert_pred

In [51]:
def create_logit_lens_viz(analysis_results, token_pos=0, layers_to_plot=[]):
    """Creates a modern heatmap visualization of predictions across model layers.
    
    Args:
        analysis_results: Analysis results dictionary
        token_pos: Position of token to analyze (default: 0)
        layers_to_plot: List of layer numbers to plot. If empty, plots all layers (default: [])
    """
    # [Previous initialization code remains the same until the trace creation]
    n_layers = max(analysis_results['layer_predictions'].keys()) + 1
    data = []
    all_values = []
    
    # Process each layer
    for layer in range(n_layers):
        # Skip layers not in layers_to_plot if it's not empty
        if layers_to_plot and layer not in layers_to_plot:
            continue
            
        row = {
            'layer': layer,
            'l_out': None,
            'expert_1': None,
            'expert_2': None,
            'expert_3': None,
            'expert_4': None,
            'expert_5': None,
            'expert_6': None,
            'shared': None,
            'predictions': {
                'l_out': [],
                'expert_1': [],
                'expert_2': [],
                'expert_3': [],
                'expert_4': [],
                'expert_5': [],
                'expert_6': [],
                'shared': []
            },
            'expert_ids': {},
            'top_tokens': {
                'l_out': '',
                'expert_1': '',
                'expert_2': '',
                'expert_3': '',
                'expert_4': '',
                'expert_5': '',
                'expert_6': '',
                'shared': ''
            }
        }
        
        # Layer predictions
        layer_preds = get_layer_predictions_for_token(analysis_results, layer, token_pos)
        if layer_preds:
            row['l_out'] = layer_preds[0][1]
            row['predictions']['l_out'] = layer_preds
            row['top_tokens']['l_out'] = layer_preds[0][0]
            all_values.append(layer_preds[0][1])
        
        # Expert predictions
        if layer in analysis_results['moe_analysis']:
            expert_preds = get_expert_preds(analysis_results, layer, token_pos)
            moe_data = analysis_results['moe_analysis'][layer]
            
            expert_weights = {}
            for pos_idx, experts in enumerate(moe_data['selected_experts']):
                if pos_idx == token_pos:
                    for expert_idx, weight in zip(experts, moe_data['expert_weights'][pos_idx]):
                        expert_weights[expert_idx] = weight
            
            # Sort experts by their top logit value instead of weights
            expert_logits = []
            for expert_idx in expert_weights:
                expert_data = next((p for e, p in expert_preds if e == expert_idx), None)
                if expert_data and expert_data[0]:
                    expert_logits.append((expert_idx, expert_data[0][1]))
            
            sorted_experts = sorted(expert_logits, key=lambda x: x[1], reverse=True)[:6]
            
            for i, (expert_idx, _) in enumerate(sorted_experts, 1):
                expert_data = next((p for e, p in expert_preds if e == expert_idx), None)
                if expert_data:
                    expert_key = f'expert_{i}'
                    row[expert_key] = expert_data[0][1]
                    row['predictions'][expert_key] = expert_data
                    row['expert_ids'][expert_key] = expert_idx
                    row['top_tokens'][expert_key] = expert_data[0][0]
                    all_values.append(expert_data[0][1])
            
            # Shared expert predictions
            shared_preds = get_shared_expert_predictions_for_token(analysis_results, layer, token_pos)
            if shared_preds:
                row['shared'] = shared_preds[0][1]
                row['predictions']['shared'] = shared_preds
                row['top_tokens']['shared'] = shared_preds[0][0]
                all_values.append(shared_preds[0][1])
        
        data.append(row)
    
    valid_values = [v for v in all_values if v is not None]
    vmin = min(valid_values) if valid_values else 0
    vmax = max(valid_values) if valid_values else 1
    
    fig = go.Figure()
    
    columns = ['l_out'] + [f'expert_{i}' for i in range(1, 7)] + ['shared']
    x_positions = list(range(len(columns)))
    
    colorscale = [
        [0, 'rgb(68,1,84)'],
        [0.25, 'rgb(59,82,139)'],
        [0.5, 'rgb(33,144,141)'],
        [0.75, 'rgb(93,201,99)'],
        [1, 'rgb(253,231,37)']
    ]
    
    # Add background squares
    for col, x_pos in zip(columns, x_positions):
        values = [row[col] for row in data]
        valid_values = [v if v is not None else float('nan') for v in values]
        
        hover_text = []
        for i, row in enumerate(data):
            preds = row['predictions'][col]
            logit_value = row[col]
            if preds:
                pred_text = "<br>".join([f"{token}: {score:.3f}" for token, score in preds])
                if col.startswith('expert_'):
                    real_id = row['expert_ids'].get(col)
                    hover_text.append(f"Layer {row['layer']} expert {real_id} (logit: {logit_value:.3f}):<br>{pred_text}")
                else:
                    hover_text.append(f"Layer {row['layer']} {col} (logit: {logit_value:.3f}):<br>{pred_text}")
            else:
                hover_text.append(f"Layer {row['layer']} {col}:<br>No data")

        # Add background squares
        fig.add_trace(go.Scatter(
            x=[x_pos] * len(values),
            y=list(range(len(values))),
            mode='markers',
            marker=dict(
                size=50,
                symbol='square',
                color=valid_values,
                colorscale=colorscale,
                showscale=True if col == 'l_out' else False,
                cmin=vmin,
                cmax=vmax,
                colorbar=dict(
                    title='logit value',
                    y=-0.15,  # Position below the plot
                    orientation='h',
                    titleside='right'
                ) if col == 'l_out' else None,
                line=dict(
                    color='rgba(255,255,255,0.2)',
                    width=1
                )
            ),
            hovertext=hover_text,
            hoverinfo='text',
            showlegend=False
        ))
        
        # Add expert IDs (top left)
        text_values = []
        text_positions = []
        for i, row in enumerate(data):
            if row[col] is not None:
                if col.startswith('expert_'):
                    text_values.append(str(row['expert_ids'].get(col, '')))
                    text_positions.append([x_pos - 0.2, i + 0.2])  # Offset for top-left position
        
        if text_values:
            fig.add_trace(go.Scatter(
                x=[p[0] for p in text_positions],
                y=[p[1] for p in text_positions],
                mode='text',
                text=text_values,
                textposition='middle center',
                textfont=dict(
                    color='white',
                    size=10
                ),
                hoverinfo='skip',
                showlegend=False
            ))
        
        # Add top tokens (center)
        text_values = []
        text_positions = []
        text_colors = []
        for i, row in enumerate(data):
            if row[col] is not None:
                text_values.append(row['top_tokens'][col])
                text_positions.append([x_pos, i])
                # Calculate normalized value between 0 and 1
                normalized_value = (row[col] - vmin) / (vmax - vmin) if vmax != vmin else 0
                # Use black text if normalized value is above 0.5 (50%)
                text_colors.append('black' if normalized_value > 0.5 else 'white')
        
        if text_values:
            for text, pos, color in zip(text_values, text_positions, text_colors):
                fig.add_trace(go.Scatter(
                    x=[pos[0]],
                    y=[pos[1]],
                    mode='text',
                    text=[text],
                    textposition='middle center',
                    textfont=dict(
                        color=color,
                        size=10
                    ),
                    hoverinfo='skip',
                    showlegend=False
                ))
    
    # Calculate dimensions
    cell_size = 50
    plot_width = 800
    plot_height = cell_size * len(data) * (plot_width / (cell_size * len(columns)))
    
    # Update layout
    fig.update_layout(
        title=dict(
            text='Expert Contributions and Predictions',
            x=0.5,
            y=0.98,
            xanchor='center',
            yanchor='top',
            font=dict(size=20, color='white')
        ),
        xaxis=dict(
            ticktext=['l_out'] + [f'expert {i}' for i in range(1, 7)] + ['shared'],
            tickvals=x_positions,
            tickangle=0,
            showgrid=False,
            zeroline=False,
            title='',
            color='white',
            range=[-0.5, len(columns) - 0.5],
            scaleanchor='y',
            scaleratio=1,
            constrain='domain'
        ),
        yaxis=dict(
            autorange='reversed',
            showgrid=False,
            zeroline=False,
            title='Layer',
            color='white',
            range=[-0.5, len(data) - 0.5],
            ticktext=[str(row['layer']) for row in data],
            tickvals=list(range(len(data))),
            constrain='domain'
        ),
        plot_bgcolor='rgb(17,17,17)',
        paper_bgcolor='rgb(17,17,17)',
        width=plot_width,
        height=plot_height + 100,  # Added extra height for colorbar
        margin=dict(l=50, r=50, t=100, b=100),  # Increased bottom margin
        hovermode='closest'
    )
    
    # Add subtle grid lines
    fig.update_xaxes(showline=True, linewidth=1, linecolor='rgba(255,255,255,0.1)')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='rgba(255,255,255,0.1)')
    
    return fig

In [53]:
l = []
x = len(tokenizer.encode(prompt))
print(x)
fig = create_logit_lens_viz(analysis, token_pos=x-1, layers_to_plot=l)
# save the fig
i = 1
while os.path.exists(f'logit_lens_viz_{i}.html'):
    i += 1
fig.write_html(f'logit_lens_viz_{i}.html')
fig.write_image(f'logit_lens_viz_{i}.png')
fig.show()

17
expert 9 : [('emon', 8.7734375), ('铃', 8.75), ('�', 8.34375), (' ', 8.3359375), ('слу', 8.0703125)]
expert 17 : [('okes', 9.0390625), ('ternoons', 8.875), ('ynam', 8.765625), ('aging', 8.6953125), (' succe', 8.671875)]
expert 32 : [('�起', 7.0234375), ('古镇', 6.66796875), ('爆', 6.5390625), ('ackle', 6.33984375), ('erred', 5.74609375)]
expert 33 : [('ascar', 9.8359375), (' Schwennicke', 8.9453125), (';->', 8.859375), ('рни', 8.828125), ('бора', 8.8046875)]
expert 37 : [(' ', 12.0859375), (' A', 10.0625), ('辞', 8.53125), (' (', 8.3828125), ('_', 8.2890625)]
expert 53 : [(' constel', 8.390625), ('och', 8.140625), ('斩', 7.85546875), (' Const', 7.61328125), ('бре', 7.56640625)]
expert 5 : [('ivet', 10.5546875), ('lir', 9.296875), ('宜', 8.96875), ('hern', 8.5859375), ('enser', 8.5234375)]
expert 13 : [('aculate', 9.0546875), ('riu', 8.65625), ('�', 8.5625), ('irmed', 8.4765625), ('letic', 8.2109375)]
expert 16 : [('orra', 9.109375), (' expulsat', 8.8359375), ('iano', 8.828125), (' Tafel', 8